![](img/1_cZKKqiASdIfCy_qvkZ0pSA.webp)

# Convolution 
+ Phép tích chập:
$$
s(t) = (x * w)(t) = \sum_a x(a)w(t - a) \ \ \ \ \ \ \ \ \ \ \ \ \text{với dữ liệu liên tục thay bằng tích phân}
$$
+ VD: Với dữ liệu ảnh 2D $I$, hạt nhân $K$:
$$
S(i, j) = (I * K)(i, j) = \sum_m \sum_n I(m, n) K(i - m, j -n) \\
S(i, j) = (K * I)(i, j) = \sum_m \sum_n I(i - m,j - n) K(i, j) \\ \ \ \ \ \ \ \ \text{phép tích chập có tính giao hoán (thường dùng công thức này)}
$$

+ Tích chập sử dụng 3 ý tưởng chính để cải thiện hệ thống học máy:
    + Sparse interactions (kết nối thưa)
    + parameter sharing
    + equivariant representation (biểu diễn tương đẳng)

+ **Sparse interactions**:
    + Các nerual network truyền thống sử dụng phép nhân ma trận
    + Cải tiến của CNN làm cho kernel có kích thước nhỏ hơn đầu vào -> lưu trữ ít tham số hơn, giảm yêu cầu về bộ nhớ và tính toán

+ **Parameter sharing**: 
    + Trong NN truyền thống, 1 ma trận trọng số chỉ sử dụng 1 lần khi tính toán đầu ra của 1 lớp
    + Trong CNN, mỗi phần tử của kernel được sử dụng trong mọi vị trí của đầu vào -> hữu ích khi phát hiện các đặc trưng xuất hiện ở nhiều vị trí. Thay ví học 1 bộ trọng số riêng biệt cho mỗi vị trí

+ **Equivariant representation**
    + Nếu đầu vào dịch chuyển đầu ra cx dịch chuyển tương ứng theo cách giống nhau
    + Ví dụ: Với hình ảnh, tích chập tạo ra feature map 2D, nếu chúng ta dịch chuyển đầu vào thì biểu diên  của nó dịch chuyển tương ứng ở đầu ra. Khi đối tượng có cạnh trong hình ảnh được di chuyển thì filter vẫn phát hiện cạnh đó nhưng ở vị trí mơi.


# Các loại Pooling
+ Giúp giảm kích thước feature map -> giảm tham số, tính toán
+ Giúp giữ lại cách đặc trưng quan trọng, giảm thiểu sự nhạy cảm với thay đổi nhỏ trong đầu vào
+ Các Pooling:
    + Max pooling: chọn ra giá trị lớn nhất trong mỗi khối nhỏ của feature map
    + Average pooling: giá trị trung bình trong mỗi khối nhỏ của feature map
    + Global pooling (global max pooling hoặc global average pooling): tính toán giá trị max hoặc trung bình cho toàn bộ feature map. Điều này thường được sử dụng trước khi kết nối với lớp fully connected để giảm kích thước đầu ra.

![](img/poolingwebp.webp)
![](img/global_pooling.webp)

# Flatten Layer

Flatten Layer là một lớp đơn giản nhưng rất quan trọng, đặc biệt khi bạn chuyển từ các lớp convolution và pooling sang các lớp fully connected.

- Cách Hoạt Động
  - **Chuyển Đổi nhiều chiều sang 1D**: Lớp flatten chuyển đổi một tensor nhiều chiều (thường là 3D với chiều cao, chiều rộng và số lượng channel) thành một vector 1D. Ví dụ, sau các lớp convolution và pooling, bạn có thể có một tensor với kích thước $ [H \times W \times C] $, trong đó $ H $ là chiều cao, $ W $ là chiều rộng và $ C $ là số lượng channel. Flatten layer sẽ chuyển đổi tensor này thành một vector với chiều dài $ H \times W \times C $.
  - **Đơn Giản Hóa Dữ Liệu**: Sau khi dữ liệu hình ảnh đã được trích xuất thành các đặc trưng, lớp flatten sẽ giúp đơn giản hóa dữ liệu để nó có thể được đưa vào lớp fully connected, nơi dữ liệu được xử lý như một vector thay vì một tensor nhiều chiều.

- Vai Trò Trong CNN
  - **Kết Nối Với Lớp Fully Connected**: Lớp Flatten là cầu nối giữa các lớp convolutional và lớp fully connected. Các lớp convolution và pooling thường giữ lại cấu trúc không gian của dữ liệu, nhưng lớp fully connected yêu cầu dữ liệu phải ở dạng vector phẳng. Do đó, Flatten layer giúp chuyển đổi định dạng này để phù hợp với lớp fully connected.
  - **Ví Dụ**: Giả sử đầu vào cho lớp Flatten là một tensor với kích thước $ [7 \times 7 \times 512] $, sau khi Flatten, đầu ra sẽ là một vector có kích thước $ [7 \times 7 \times 512 = 25088] $. Vector này có thể được đưa vào một lớp fully connected.


# Fully Connected Layer

- Cách Hoạt Động
  - Công thức toán học cho một lớp fully connected là:
    $$
    z = W \cdot x + b
    $$
    Trong đó:
    - $ x $ là đầu vào (vector đặc trưng từ lớp trước).
    - $ W $ là ma trận trọng số
    - $ b $ là vector bias
    - $ z $ là đầu ra của lớp fully connected, thường sau đó được đưa qua activation function như ReLU, Sigmoid, hoặc Softmax.


# Ví dụ

Xây dựng mô hình CNN phân loại xe

In [1]:
from torch import nn
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torch.optim as optim
from torchvision.datasets import ImageFolder
from torchvision import transforms
import torch.nn.functional as F

In [2]:
train_dir = 'vehicle/Training'
test_dir = 'vehicle/Testing'

train_dataset = ImageFolder(train_dir, transform=transforms.Compose([transforms.Resize((128,128)), transforms.ToTensor()]))
test_dataset = train_dataset = ImageFolder(test_dir, transform=transforms.Compose([transforms.Resize((128,128)), transforms.ToTensor()]))

In [3]:
img, label = train_dataset[0]
img.shape

torch.Size([3, 128, 128])

In [4]:
train_dataset.classes

['bus', 'car', 'moto', 'pedestrian', 'truck']

In [5]:
batch_size = 32

val_size = int(len(train_dataset) * 0.3)
train_size = len(train_dataset) - val_size
train_data, val_data = random_split(train_dataset, [train_size, val_size])

train_dl = DataLoader(train_data, batch_size, shuffle=True)
val_dl = DataLoader(val_data, batch_size, shuffle=True)
test_dl = DataLoader(test_dataset, batch_size=64)

In [6]:
class MyCNN(nn.Module):
    def __init__(self ):
        super(MyCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2)

        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride = 1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2)

        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride = 1)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2)

        self.fl = nn.Flatten()

        self.fc1 = nn.Linear(64 * 14 * 14, 1024)
        self.relu4 = nn.ReLU()
        self.fc2 = nn.Linear(1024, 256)
        self.relu5 = nn.ReLU()
        self.fc3 = nn.Linear(256, 5)

    def forward(self, x):
        out = self.conv1(x)
        out = self.relu1(out)
        out = self.pool1(out)

        out = self.conv2(out)
        out = self.relu2(out)
        out = self.pool2(out)

        out = self.conv3(out)
        out = self.relu3(out)
        out = self.pool3(out)

        out = self.fl(out)

        out = self.fc1(out)
        out = self.relu4(out)
        out = self.fc2(out)
        out = self.relu5(out)
        out = self.fc3(out)

        return out


In [7]:
def train(model, train_dataloder, val_dataloader,  num_epochs, opt_func = optim.Adam, lr = 0.01):
    optimizer = opt_func(model.parameters(), lr)
    criterion = nn.CrossEntropyLoss()

    train_losses = []
    val_losses = []

    for i in range(num_epochs):
        model.train()

        train_loss_epoch = 0.0
        val_loss_epoch = 0.0

        for batch_X, batch_y in train_dataloder:

            output = model(batch_X)
            loss = criterion(output, batch_y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss_epoch += loss.item()

        model.eval()
        with torch.no_grad():
            for X_val, y_val in val_dataloader:
                output = model(X_val)
                loss = F.cross_entropy(output, y_val)
                val_loss_epoch += loss.item()

        avg_train_loss = train_loss_epoch / len(train_dataloder)
        train_losses.append(avg_train_loss)

        avg_val_loss = val_loss_epoch / len(val_dataloader)
        val_losses.append(avg_val_loss)

        print(f"Epoch {i + 1} / {num_epochs}: train loss: {avg_train_loss}, val loss: {avg_val_loss}")

    return train_losses, val_losses

In [11]:
model = MyCNN()
train_losses, val_losses = train(model, train_dl, val_dl, 100)

Epoch 1 / 100: train loss: 7.689721012115479, val loss: 1.5368870496749878
Epoch 2 / 100: train loss: 1.6073374509811402, val loss: 1.5111468434333801
Epoch 3 / 100: train loss: 1.5897489547729493, val loss: 1.6547085046768188
Epoch 4 / 100: train loss: 1.626796817779541, val loss: 1.550927221775055
Epoch 5 / 100: train loss: 1.5578304767608642, val loss: 1.5093839764595032
Epoch 6 / 100: train loss: 1.6439351797103883, val loss: 1.5007765293121338
Epoch 7 / 100: train loss: 1.590745759010315, val loss: 1.5467860698699951
Epoch 8 / 100: train loss: 1.6070869445800782, val loss: 1.5539231300354004
Epoch 9 / 100: train loss: 1.5667400121688844, val loss: 1.5610191226005554
Epoch 10 / 100: train loss: 1.5721670627593993, val loss: 1.5497498512268066
Epoch 11 / 100: train loss: 1.6000787258148192, val loss: 1.5291836857795715
Epoch 12 / 100: train loss: 1.6770551681518555, val loss: 1.5149553418159485
Epoch 13 / 100: train loss: 1.5438856840133668, val loss: 1.524292528629303
Epoch 14 / 10

In [13]:
test_dl = DataLoader(test_dataset, batch_size=(len(test_dataset)), shuffle=False)
model.eval()


with torch.no_grad():
    for batch_X, batch_y in test_dl:    
        outputs = model(batch_X)
        _, predicted = torch.max(outputs, 1)

        accuracy = (predicted == batch_y).sum().item() / batch_y.size(0)
        print(f'Accuracy test: {accuracy:.4f}')

Accuracy test: 0.9508
